# ML Final Project


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
## Imports
import numpy as np
# from sklearn.linear_model import train_test_split

In [ ]:
## Load data
path = '/content/drive/MyDrive/ML Final Project/data/'
data_amzn = np.loadtxt(path + 'amazon_cells_labelled.txt', dtype='object',delimiter='\t')
x_amzn = [data[0] for data in data_amzn]
y_amzn = [int(data[1]) for data in data_amzn]

data_imdb = np.loadtxt(path + 'imdb_labelled.txt', dtype='object',delimiter='\t')
x_imdb = [data[0] for data in data_imdb]
y_imdb = [int(data[1]) for data in data_imdb]

data_yelp = np.loadtxt(path + 'yelp_labelled.txt', dtype='object',delimiter='\t')
x_yelp = [data[0] for data in data_yelp]
y_yelp = [int(data[1]) for data in data_yelp]

train_cutoff = 800
dev_cutoff = 900

x_train = x_amzn[0:train_cutoff] + x_imdb[0:train_cutoff] + x_yelp[0:train_cutoff]
y_train = y_amzn[0:train_cutoff] + y_imdb[0:train_cutoff] + y_yelp[0:train_cutoff]

x_dev = x_amzn[train_cutoff:dev_cutoff] + x_imdb[train_cutoff:dev_cutoff] + x_yelp[train_cutoff:dev_cutoff]
y_dev = y_amzn[train_cutoff:dev_cutoff] + y_imdb[train_cutoff:dev_cutoff] + y_yelp[train_cutoff:dev_cutoff]

x_test = x_amzn[dev_cutoff:] + x_imdb[dev_cutoff:] + x_yelp[dev_cutoff:]
y_test = y_amzn[dev_cutoff:] + y_imdb[dev_cutoff:] + y_yelp[dev_cutoff:]

In [ ]:
## Preprocess data
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(x_train)
X_dev = vectorizer.transform(x_dev)
X_test = vectorizer.transform(x_test)


In [ ]:
import tensorflow as tf
from keras.preprocessing import sequence

vocab_size = 10000
encoded_training = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in x_train] 
encoded_dev = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in x_dev]
encoded_test = [tf.keras.preprocessing.text.one_hot(d, vocab_size) for d in x_test]
max_words = 500

X_train = sequence.pad_sequences(encoded_training, maxlen=max_words)
X_dev = sequence.pad_sequences(encoded_dev, maxlen=max_words)
X_test = sequence.pad_sequences(encoded_test, maxlen= max_words)


In [ ]:
print(encoded_training)

[[1324, 575, 536, 365, 98, 765, 853, 1691, 599, 347, 2405, 451, 2405, 599, 909, 1781, 1556, 2261, 1903, 2111, 379], [1839, 2629, 1608, 108], [2140, 765, 599, 2007], [1272, 1691, 2801, 765, 1048, 54, 573, 2299, 2500, 12, 377, 2248], [599, 1427, 536, 2140], [1556, 1716, 1691, 2797, 599, 599, 1691, 2015, 347, 1691, 1434, 1094, 2793, 1691, 2015, 2858, 292], [445, 302, 1716, 1538, 1921, 471, 1538, 2210, 609, 1432, 492, 599, 469, 1102, 246, 84, 1102, 522, 1905, 1903, 1905], [445, 302, 2641, 1583, 2121, 302, 85, 1716, 685], [1552, 1691, 468, 1556, 1668, 1264, 231], [79, 2111, 1929, 1102, 231, 1651, 1393], [1651, 599, 1294, 875, 536, 2140], [2711, 929, 673, 1496, 1896, 1844, 566, 599, 1559, 2897, 1691, 599, 2246, 2897], [445, 599, 2545, 763, 163, 1903, 2111, 900, 557, 1957, 1556, 2537, 1691, 2706, 1049, 1808, 1651, 2406, 1294, 566, 599, 2065], [673, 1839, 875, 1288], [599, 2907, 536, 673, 774, 2396, 599, 1624, 204, 536, 2704, 673, 439, 193, 157], [647, 640, 765, 777, 1905, 2204, 1355, 2111, 65

In [61]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

from tensorflow import keras
from tensorflow.keras import layers

embedding_size=64
model=Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=max_words))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [62]:
opt = keras.optimizers.Adam(learning_rate=0.011)

model.compile(loss='binary_crossentropy', 
             optimizer=opt, 
             metrics=['accuracy'])

In [63]:
batch_size = 60
num_epochs = 3

y_dev = np.array(y_dev)
y_train = np.array(y_train)



model.fit(X_train, y_train, validation_data=(X_dev, y_dev), batch_size=batch_size, epochs=num_epochs)

Epoch 1/3
40/40 [==============================] - 22s 545ms/step - loss: 0.6135 - accuracy: 0.6771 - val_loss: 0.4892 - val_accuracy: 0.7667
Epoch 2/3
40/40 [==============================] - 22s 544ms/step - loss: 0.2368 - accuracy: 0.9079 - val_loss: 0.5653 - val_accuracy: 0.7367
Epoch 3/3
40/40 [==============================] - 22s 548ms/step - loss: 0.0992 - accuracy: 0.9692 - val_loss: 0.7212 - val_accuracy: 0.7833


In [ ]:
model.evaluate(X_test, np.array(y_test))

10/10 [==============================] - 1s 77ms/step - loss: 1.0651 - accuracy: 0.8060


[1.0650864839553833, 0.8060200810432434]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# print(len(y_test))

predictions = model.predict(X_test)  
# print(predictions)

predictions = np.where(predictions > .5,1,0)

# print(confusion_matrix(np.array(y_test), predictions))
print(classification_report(np.array(y_test), predictions))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85       186
           1       0.74      0.79      0.76       113

    accuracy                           0.82       299
   macro avg       0.80      0.81      0.81       299
weighted avg       0.82      0.82      0.82       299

